In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# splinter stuff 
def startSplinter(url):
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)
    return browser

In [3]:
browser = startSplinter('https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/ryaneccleston-murdock/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


### Nasa Articles Scrape

In [4]:
nasa_results = {'news_titles': [],
                'p_text': []}
            
html = browser.html
pg_soup = BeautifulSoup(html, 'html')
articles = pg_soup.find_all('div', class_='list_text')
for article in articles:
    try: 
        nasa_results['news_titles'].append(article.find('div', class_='content_title').text.strip())
        nasa_results['p_text'].append(article.find('div', class_='article_teaser_body').text.strip())
    except AttributeError:
        print('Nothing to add apparently!')
browser.quit()

In [5]:
nasa_results

{'news_titles': ["NASA's Mars Perseverance Rover Provides Front-Row Seat to Landing, First Audio Recording of Red Planet",
  'NASA to Reveal New Video, Images From Mars Perseverance Rover',
  "NASA's Mars Helicopter Reports In",
  "NASA's Perseverance Rover Sends Sneak Peek of Mars Landing",
  "Touchdown! NASA's Mars Perseverance Rover Safely Lands on Red Planet",
  "Searching for Life in NASA's Perseverance Mars Samples",
  "The Mars Relay Network Connects Us to NASA's Martian Explorers",
  "NASA's Next Mars Rover Is Ready for the Most Precise Landing Yet",
  "Sensors Prepare to Collect Data as Perseverance Enters Mars' Atmosphere",
  'InSight Is Meeting the Challenge of Winter on Dusty Mars',
  'NASA Invites Public to Share Thrill of Mars Perseverance Rover Landing',
  'Tricky Terrain: Helping to Assure a Safe Rover Landing',
  'Where Should Future Astronauts Land on Mars? Follow the Water',
  "NASA's Perseverance Pays Off Back Home",
  'Could the Surface of Phobos Reveal Secrets of 

### Feature Image Scrape

In [6]:
browser = startSplinter('https://www.jpl.nasa.gov/images?query=mars')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/ryaneccleston-murdock/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [7]:
browser.find_by_css('.text-theme-red')[1].click() # finds and clicks the 'SORT BY' menu
browser.find_by_tag('option')[2].click() # finds and clicks the 'LATEST' option
browser.find_by_css('.SearchResultCard').first.click() # finds and clicks first/featured image

In [ ]:
# On featured image page
# PAGE CHANGE - LET PAGE FULLY LOAD BEFORE RUNNING

featured_img_html = browser.html
img_soup = BeautifulSoup(featured_img_html, 'html')
featured_img_url = img_soup.find_all('img', class_='BaseImage object-scale-down')[0]['data-src']
print(featured_img_url)
browser.quit()

### Mars Facts Scrape (with Pandas)

In [ ]:
mars_facts_table = pd.read_html('https://space-facts.com/mars/')[0]

In [ ]:
# mars_facts_html = browser.html
# mars_facts_soup = BeautifulSoup(mars_facts_html, 'html')

# for row in mars_facts_soup.find_all('tr'):
#     print(row.text.split(':'))

In [ ]:
mars_facts_table.iloc[:,0] = mars_facts_table.iloc[:,0].str.strip(':')
mars_facts_table = mars_facts_table.rename(columns={0: 'Fact Topics', 1: 'Fact Value'})
mars_facts_table

In [ ]:
mars_facts_table_html = mars_facts_table.to_html().replace('\n', '')

### Mars Hemisphere Scrape

In [15]:
browser = startSplinter('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/ryaneccleston-murdock/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [ ]:
# find and click hem image (for however many)
# find and click open 
# save hem img title and link in dict per img, save each dict to list
# return to home 

In [16]:
hemisphere_image_urls = []

for i in range(0,4):
    browser.find_by_css('.thumb')[i].click() # clicks on hemisphere button on Homepage

    # Image page actions 
    browser.find_by_text('Open').click()
    img_page_html = browser.html
    img_page_soup = BeautifulSoup(img_page_html, 'html')
    img = img_page_soup.find_all('img', class_='wide-image')[0]
    title = img_page_soup.find('h2', class_='title').text.strip(' Enhanced') + 'e'
    img_url = 'https://astrogeology.usgs.gov' + img['src']

    hemisphere_image_urls.append({title: img_url})
    browser.back()
browser.quit()

In [ ]:
hemisphere_image_urls